<a href="https://colab.research.google.com/github/wgova/time_series_trade/blob/master/data_prep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install -q oec
import oec
import pandas as pd
import numpy as np
import os, os.path, csv, requests, pathlib
import math
from datetime import datetime
import matplotlib.pyplot as plt
%matplotlib inline

In [0]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

## OED data API functions

In [0]:
def build_call(*args):
    call_url = 'http://atlas.media.mit.edu/'
    for val in args:
        call_url += str(val) + '/'
    return call_url


def request_data(call_url):
    r = requests.get(call_url)
    response_dict = r.json()
    json_list = response_dict['data']  # list of dicts containing data
    return json_list


def get_countries(filename=None):
    call = build_call('attr', 'country')
    json_list = request_data(call)
    if filename is not None:
        data_to_csv(json_list, filename)
    return json_list


def get_products(classification, filename=None):
    call = build_call('attr', classification)
    json_list = request_data(call)
    if filename is not None:
        data_to_csv(json_list, filename)
    return json_list


def get_trade(classification, trade_flow, year, origin, destination,product, filename=None):
    call = build_call(classification, trade_flow, year, origin, destination,
                      product)
    json_list = request_data(call)
    if filename is not None:
        data_to_csv(json_list, filename)
    return json_list


def trade_params(classification, trade_flow, year, origin, destination,product):
    parameters = {'classification': classification,
                  'trade_flow': trade_flow,
                  'year': year,
                  'origin': origin,
                  'destination': destination,
                  'product': product}
    return parameters


def get_header(json_list):
    header = set()
    for dict in json_list:
        header.update(dict.keys())
    header = list(header)
    header.sort()
    return header


def create_csv(json_list, filename):
    with open(filename, 'w') as csvfile:
        header = get_header(json_list)
        cw = csv.writer(csvfile)
        cw.writerow(header)
        for dict in json_list:
            row = dict_to_list(dict, header)
            cw.writerow(row)


def dict_to_list(dict, header):
    row = []
    for field in header:
        if field in dict:
            row.append(str(dict[field]))
        else:
            row.append(None)
    return row

## Generate dataset

In [0]:
trade_codes = ['6519','6531','8471','6571','7849','7810','7842','7764','7723','7522','2924','5419','5417','7763','7711','7188']
PATH = '/content/drive/My Drive/Stellenbosch/Webster'
os.chdir(PATH)
trade_classification = 'sitc'
def countries_and_products():
  countries = oec.get_countries()
  products = oec.get_products(trade_classification)
  create_csv(countries,f'{PATH}/list_countries.csv')
  create_csv(products,f'{PATH}/list_products.csv')

In [0]:
countries_and_products()

In [0]:
import os.path
from os import path
def download_data(list_of_codes):
  for i in trade_codes:
    csv_name = f"{trade_classification}-{i}"
    # Set parameters to extract top exporters from the API documentation
    params = {'classification': trade_classification,
          'trade_flow': 'export',
          'year': 'all',
          'origin': 'show', #Set origin to all for comparison
          'destination': 'all',
          'product': i} #Set code for product of interest
    print(f'Data for {i} processed.....')
    oec_data = get_trade(**params)
    # Save the results in CSV file
    if path.exists(PATH):
      create_csv(oec_data,f'{csv_name}.csv')#,locals(),globals())
    else:
      print('Choose destination directory')

In [0]:
download_data(trade_codes)

In [0]:
import pandas as pd
import numpy as np
import os, os.path, csv, requests, pathlib
import math
from datetime import datetime

PATH = '/content/drive/My Drive/Stellenbosch/Webster'
include_list = ['year','origin_id','sitc_id','export_val','export_rca']
def create_df(path):
  trade_data = [i for i in os.listdir(path) if i.startswith('sitc') and '.csv' in i]
  trade_history = []
  for file in trade_data:
    file_name = pd.read_csv(file,usecols=include_list,parse_dates=['year'])#, index_col=['year'])
    trade_history.append(file_name)
  products_dataframe = pd.concat(trade_history)
  return products_dataframe

In [0]:
trade_df = create_df(PATH)

In [0]:
country_list = pd.read_csv('list_countries.csv',usecols=['id','name'],
                        na_values='',na_filter=True)
country_list.rename(columns=lambda x: x.replace('name','country'), inplace=True)
country_list.rename(columns=lambda x: x.replace('id','origin_id'), inplace=True)
country_list.info()

In [0]:
trade_dframe = trade_df.merge(country_list, on=['origin_id'],how='inner')
trade_dframe['sitc_id'].astype('category')
trade_dframe.fillna(0,inplace=True)
trade_dframe.head(2)

In [0]:
products = pd.read_csv(f'{PATH}/list_products.csv',usecols=['id','name','display_id'])
products.rename(columns={'id':'sitc_id'},inplace=True)
products = products[products['display_id'].isin(trade_codes)]
products['name'] = products.name.astype('category')
products['sitc_id'] = products.sitc_id.astype(int)
trade_dframe['product_name'] = trade_dframe.sitc_id.map(products.set_index('sitc_id')['name'].to_dict())
products.info()

In [0]:
agg_product = pd.pivot_table(trade_dframe, index='year', columns='product_name', values='export_val', aggfunc=np.mean)
agg_product.fillna(0,inplace=True)

In [0]:
# agg_product.rename(columns=products.set_index('sitc_id')['name'], inplace=True)
# agg_product.rename(columns=prod_dict,inplace=True)
agg_product.head()

In [0]:
prod_dict = {'Yarn of Textile Fibers':'yarn_fiber',
                             'Synthetic Woven Fabrics':'synthetic_fiber',
                             'Textile Fabrics Clothing Accessories':'accessory_fabrics',
                             'Vehicles Parts and Accessories':'vehicle_parts',
                             'Personal Computers':'computers',
                             'Pharmaceutical Flora':'pharma_flora',
                             'Non-Medicinal Pharmaceutical Products':'non_medicinals',
                             'Diodes, Transistors and Photocells':'diodes_photocells',
                             'Miscellaneous Engines':'engines'
                             }
agg_product.rename(columns=prod_dict,inplace=True)
def clean_header(df):
  df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')
  return df.columns
clean_header(agg_product)
# trade_dframe[cols].head()
agg_product.head()

In [0]:
agg_country = pd.pivot_table(trade_dframe, index='year', columns='country', values='export_val', aggfunc=np.mean)
agg_south_africa = pd.pivot_table(trade_dframe[trade_dframe.country=="South Africa"], index='year', columns='product_name', values='export_val', aggfunc=np.mean)
agg_south_africa.fillna(0,inplace=True)
agg_south_africa.rename(columns=prod_dict,inplace=True)
clean_header(agg_south_africa)

In [0]:
plt.figure(figsize=(24, 8))
plt.subplot(131)
plt.plot(agg_product) #/ agg_year_item.mean(0)[np.newaxis])
plt.title("Global value of trade per product")
plt.xlabel("Year")
plt.legend(agg_product.columns,loc="upper left")
plt.ylabel("Export val")

plt.subplot(133)
plt.plot(agg_country) #/ agg_year_store.mean(0)[np.newaxis])
plt.title("Value of trade in basket per country")
plt.xlabel("Year")
plt.ylabel("Export value")

plt.subplot(132)
plt.plot(agg_south_africa) #/ agg_year_store.mean(0)[np.newaxis])
plt.title("Value of trade in basket for South Africa")
plt.xlabel("Year")
plt.legend(agg_south_africa.columns,loc="upper left")
plt.ylabel("Export value")
plt.show()

In [0]:
agg_south_africa.plot(subplots=True,figsize=(12, 16),title="South Africa product basket")
plt.show()

In [0]:
# https://pbpython.com/pandas-pivot-table-explained.html
agg_country_product = pd.pivot_table(trade_dframe[trade_dframe.product_name==id], index='year', columns='country', values='export_val', aggfunc=np.mean,fill_value=0)
agg_country_product.head()

In [0]:
for id in trade_dframe.product_name.unique():
  product = pd.pivot_table(trade_dframe[trade_dframe.product_name==id], index='year', columns='country', values='export_val', aggfunc=np.mean)
  product.to_csv(f'{id}_sitc.csv')
  plt.plot(product)
  plt.title(f"Exports for {id} by country")
  plt.xlabel("Year")
  plt.ylabel("Export value")
  plt.show()

### Plotting functions (TODO)

In [0]:
# def plot_tile(result, ax, country_name):
    
#     ax.set_title(f"{country_name}")
    
#     # Colors
#     ABOVE = [1,0,0]
#     MIDDLE = [1,1,1]
#     BELOW = [0,0,0]
#     cmap = ListedColormap(np.r_[
#         np.linspace(BELOW,MIDDLE,25),
#         np.linspace(MIDDLE,ABOVE,25)
#     ])
#     color_mapped = lambda y: np.clip(y, .5, 1.5)-.5
    
#     index = result['ML'].index.get_level_values('date')
#     values = result['ML'].values
    
#     # Plot dots and line
#     ax.plot(index, values, c='k', zorder=1, alpha=.25)
#     ax.scatter(index,values,s=40,lw=.5,c=cmap(color_mapped(values)),edgecolors='k', zorder=2)
    
#     # Aesthetically, extrapolate credible interval by 1 day either side
#     extended = pd.date_range(start=pd.Timestamp('2020-03-01'),
#                              end=index[-1]+pd.Timedelta(days=1))
    
#     ax.fill_between(extended,lowfn(date2num(extended)),
#                     highfn(date2num(extended)),
#                     color='k',alpha=.1,
#                     lw=0,zorder=3)

#     ax.axhline(1.0, c='k', lw=1, label='Add label', alpha=.25);
    
#     # Formatting
#     ax.xaxis.set_major_locator(mdates.MonthLocator())
#     ax.xaxis.set_major_formatter(mdates.DateFormatter('%b'))
#     ax.xaxis.set_minor_locator(mdates.DayLocator())
#     ax.yaxis.set_major_locator(ticker.MultipleLocator(1))
#     ax.yaxis.set_major_formatter(ticker.StrMethodFormatter("{x:.1f}"))
#     ax.yaxis.tick_right()
#     ax.spines['left'].set_visible(False)
#     ax.spines['bottom'].set_visible(False)
#     ax.spines['right'].set_visible(False)
#     ax.margins(0)
#     ax.grid(which='major', axis='y', c='k', alpha=.1, zorder=-2)
#     ax.margins(0)
#     ax.set_ylim(0.0,3.5)
#     ax.set_xlim(pd.Timestamp('1960'), result.index.get_level_values('date')[-1]+pd.Timedelta(days=1))
#     fig.set_facecolor('w')

In [0]:
## Requires df.melt() for 2D plot
# origin = trade_dframe.country.unique()
# vals = trade_dframe[['country','export_val']]
# ncols = 5
# nrows = int(np.ceil(len(trade_dframe) / ncols))

# fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(15, nrows*3))

# for i, (origin, vals) in enumerate(vals.items()):
#     if (len(result) >= 2):
#       plot_rt(vals, axes.flat[i], origin)
#     else:
#       break

# fig.tight_layout()
# fig.set_facecolor('w')

# Extract comprehensive features

### Packages

### TFRESH

- tsfresh.feature_extraction.settings.ComprehensiveFCParameters: includes all features without parameters and all features with parameters, each with different parameter combinations. This is the default for extract_features if you do not hand in a default_fc_parameters at all.
- tsfresh.feature_extraction.settings.MinimalFCParameters: includes only a handful of features and can be used for quick tests. The features which have the “minimal” attribute are used here.
- tsfresh.feature_extraction.settings.EfficientFCParameters: Mostly the same features as in the tsfresh.feature_extraction.settings.
- ComprehensiveFCParameters, but without features which are marked with the “high_comp_cost” attribute. This can be used if runtime performance plays a major role.

In [0]:
# !pip install -q tsfresh 
import os
import logging
import warnings

import pandas as pd
import matplotlib.pyplot as plt
from pandas_datareader.data import DataReader as pdr
import fix_yahoo_finance as yf
yf.pdr_override()

from tsfresh import extract_features, select_features
from tsfresh import defaults
from tsfresh.feature_extraction import feature_calculators
from tsfresh.feature_extraction.settings import ComprehensiveFCParameters,MinimalFCParameters,EfficientFCParameters
from tsfresh.utilities import dataframe_functions, profiling
from tsfresh.utilities.distribution import MapDistributor, MultiprocessingDistributor,DistributorBaseClass
from tsfresh.utilities.string_manipulation import convert_to_output_format

import tsfresh
from tsfresh.feature_extraction.settings import EfficientFCParameters
from tsfresh.utilities.dataframe_functions import roll_time_series

%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

def extract_product_features(df,fc_parameter):
  features_product = []
  extraction_method = fc_parameter.__class__.__name__
  for p in df.sitc_id.unique():
    product = df[df.sitc_id==p]
    p_features = extract_features(product[["export_val","year","country"]],
                                  column_id="country",
                                  column_sort="year",
                                  column_value=None,column_kind=None,
                                  chunksize=None,
                                  default_fc_parameters=fc_parameter)
    features_product.append(p_features)
    p_features.to_csv(f"{p}_{extraction_method}_expval.csv")
  print(f'Fatures for {p} extracted.....{len(features_product)}')
  return p_features, features_product

stats_features = {
    'length': None,
    'maximum': None,
    'mean': None,
    'median': None,
    'minimum': None,
    'standard_deviation': None,
    'sum_values': None,
    'variance': None,
    'skewness': None,
    'kurtosis': None,
    }
{ 'skewness': None,
 'standard_deviation': None,}

MinimalFCParameters = {'length': None,
 'maximum': None,
 'mean': None,
 'median': None,
 'minimum': None,
 'standard_deviation': None,
 'sum_values': None,
 'variance': None}

%timeit
# fc_parameters=[MinimalFCParameters(),EfficientFCParameters(),ComprehensiveFCParameters()]
# extract_product_features(trade_dframe,stats_features)

In [0]:
EfficientFCParameters()

### Extract features

In [0]:
#EfficientFCParameters()
features_to_review = {'abs_energy': None,
  {'attr': 'slope', 'chunk_len': 10, 'f_agg': 'max'},
  {'attr': 'slope', 'chunk_len': 10, 'f_agg': 'min'},
 'augmented_dickey_fuller': [{'attr': 'teststat'},{'attr': 'pvalue'},{'attr': 'usedlag'}],
 'autocorrelation': [{'lag': 0},{'lag': 1},{'lag': 2},{'lag': 3},{'lag': 4},{'lag': 5},{'lag': 6},{'lag': 7},{'lag': 8},{'lag': 9}],
 'binned_entropy': [{'max_bins': 10}],
 'count_above': [{'t': 0}],
 'count_above_mean': None,
 'count_below_mean': None,
 'fft_aggregated': [{'aggtype': 'centroid'},{'aggtype': 'variance'},{'aggtype': 'skew'},{'aggtype': 'kurtosis'}],
 'first_location_of_maximum': None,
 'first_location_of_minimum': None,
 'friedrich_coefficients': [{'coeff': 0, 'm': 3, 'r': 30},kurtosis': None,
 'large_standard_deviation': [{'r': 0.05},{'r': 0.1}],
  'last_location_of_maximum': None,
  'last_location_of_minimum': None,
  'length': None,
  'linear_trend': [{'attr': 'pvalue'},{'attr': 'rvalue'},{'attr': 'intercept'},{'attr': 'slope'},{'attr': 'stderr'}],
  'linear_trend_timewise': [{'attr': 'pvalue'},{'attr': 'rvalue'},{'attr': 'intercept'},{'attr': 'slope'},{'attr': 'stderr'}],
  'maximum': None,
  'mean': None,
  'mean_abs_change': None,
  'mean_change': None,
  'mean_second_derivative_central': None,
  'median': None,
  'minimum': None,
  'partial_autocorrelation': [{'lag': 0},{'lag': 1},{'lag': 2},{'lag': 3},{'lag': 4},{'lag': 5},{'lag': 6},{'lag': 7},{'lag': 8},{'lag': 9}],
  'quantile': [{'q': 0.1},{'q': 0.2},{'q': 0.3},{'q': 0.4},{'q': 0.6},{'q': 0.7},{'q': 0.8},{'q': 0.9}],
  'range_count': [{'max': 1, 'min': -1},{'max': 0, 'min': 1000000000000.0},{'max': 1000000000000.0, 'min': 0}],
 'ratio_beyond_r_sigma': [{'r': 0.5},{'r': 1},{'r': 1.5},{'r': 2},{'r': 2.5},{'r': 3},{'r': 5},{'r': 6},{'r': 7},{'r': 10}],
 'ratio_value_number_to_time_series_length': None,
 'skewness': None,
 'spkt_welch_density': [{'coeff': 2}, {'coeff': 5}, {'coeff': 8}],
 'standard_deviation': None,
 'sum_of_reoccurring_data_points': None,
 'sum_of_reoccurring_values': None,
 'sum_values': None,
 'symmetry_looking': [{'r': 0.0},{'r': 0.05},{'r': 0.1},{'r': 0.15000000000000002},{'r': 0.2},{'r': 0.25},{'r': 0.30000000000000004},
                      {'r': 0.35000000000000003},{'r': 0.4},{'r': 0.45},{'r': 0.5},{'r': 0.55},{'r': 0.6000000000000001},
                      {'r': 0.65},{'r': 0.7000000000000001},{'r': 0.75},{'r': 0.8},{'r': 0.8500000000000001},{'r': 0.9},{'r': 0.9500000000000001}],
 'time_reversal_asymmetry_statistic': [{'lag': 1}, {'lag': 2}, {'lag': 3}],
 'variance': None,
 'variance_larger_than_standard_deviation': None,
 'variation_coefficient': None}
}